In [1]:
import pandas as pd
import numpy as np

In [2]:
# Import DC Crashes dataset, view all 66 columns
# ___ records in original dataset 
crashes = pd.read_csv("crashes.csv")

pd.set_option('display.max_columns', None)

print(crashes.info())
print(crashes.columns)
print(len(crashes.columns))

/var/folders/rl/6mrqzg_d1cvg3699yh85hvqm0000gp/T/ipykernel_25016/359834761.py:3: DtypeWarning: Columns (3,18) have mixed types. Specify dtype option on import or set low_memory=False.
  crashes = pd.read_csv("crashes.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338318 entries, 0 to 338317
Data columns (total 66 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   X                           338318 non-null  float64
 1   Y                           338318 non-null  float64
 2   CRIMEID                     338318 non-null  int64  
 3   CCN                         338318 non-null  object 
 4   REPORTDATE                  336949 non-null  object 
 5   ROUTEID                     338318 non-null  object 
 6   MEASURE                     338318 non-null  float64
 7   OFFSET                      338318 non-null  float64
 8   STREETSEGID                 215905 non-null  float64
 9   ROADWAYSEGID                215905 non-null  float64
 10  FROMDATE                    338252 non-null  object 
 11  TODATE                      0 non-null       float64
 12  ADDRESS                     338236 non-null  object 
 13  LATITUDE      

In [3]:
# Determine obvious columns to drop (Too unique / unnecessary for desired analysis / redundant)
# 29 total columns to be dropped --> should end with 38
drop = ['X', 'Y', 'TODATE', 'CRIMEID', 'CCN', 'ROUTEID', 'MEASURE', 'OFFSET', 'STREETSEGID', 'ROADWAYSEGID', 'XCOORD', 'YCOORD', 
        'EVENTID', 'MAR_ADDRESS', 'MAR_SCORE', 'NEARESTINTROUTEID', 'INTAPPROACHDIRECTION', 'LOCATIONERROR', 
        'LASTUPDATEDATE', 'MPDLATITUDE', 'MPDLONGITUDE', 'MPDGEOX', 'MPDGEOY', 'MAR_ID', 'BLOCKKEY', 'SUBBLOCKKEY', 
        'CORRIDORID', 'NEARESTINTKEY', 'OBJECTID']
print("# of Columns to Drop:", len(drop))
crashes = crashes.drop(columns=drop)

print("# of Columns Remaining:", len(crashes.columns))

# of Columns to Drop: 29
# of Columns Remaining: 37


Address Discrepancy between FROMDATE & REPORTDATE: 
- It appears FROMDATE more accurately represents when the crash occurred
- REPORTDATE represents when the crash was reported, and should be used as a backup

Next, create year, month, weekday columns based on crash_datetime

In [4]:
# Ensure both columns are datetime
crashes["FROMDATE"] = pd.to_datetime(crashes["FROMDATE"], errors="coerce")
crashes["REPORTDATE"] = pd.to_datetime(crashes["REPORTDATE"], errors="coerce")

# Drop rows where both FROMDATE and REPORTDATE are missing
crashes = crashes.dropna(subset=["FROMDATE", "REPORTDATE"])

# Create a new crash_datetime column
# Use FROMDATE if available, otherwise use REPORTDATE
crashes["crash_datetime"] = crashes["FROMDATE"].fillna(crashes["REPORTDATE"])

# Extract the year, month, and weekday
crashes["year"] = crashes["crash_datetime"].dt.year
crashes["month"] = crashes["crash_datetime"].dt.month
crashes["weekday"] = crashes["crash_datetime"].dt.day_name() 

Address missing values in ADDRESS/NEARESTINTSTREETNAME columns
Treat NaN values in MAJORINJURIESOTHER/MINORINJURIESOTHER/UNKNOWNINJURIESOTHER columns as 0

In [5]:
# Drop rows where either ADDRESS or NEARESTINTSTREETNAME is missing
crashes = crashes.dropna(subset=["ADDRESS", "NEARESTINTSTREETNAME"])

# Ensure "Other Injuries" columns have no NaN values
cols_to_fix = ["MAJORINJURIESOTHER", "MINORINJURIESOTHER", "UNKNOWNINJURIESOTHER"]
crashes[cols_to_fix] = crashes[cols_to_fix].fillna(0)

In [6]:
crashes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 331049 entries, 0 to 338315
Data columns (total 41 columns):
 #   Column                      Non-Null Count   Dtype              
---  ------                      --------------   -----              
 0   REPORTDATE                  331049 non-null  datetime64[ns, UTC]
 1   FROMDATE                    331049 non-null  datetime64[ns, UTC]
 2   ADDRESS                     331049 non-null  object             
 3   LATITUDE                    331049 non-null  float64            
 4   LONGITUDE                   331049 non-null  float64            
 5   WARD                        331047 non-null  object             
 6   MAJORINJURIES_BICYCLIST     331049 non-null  int64              
 7   MINORINJURIES_BICYCLIST     331049 non-null  int64              
 8   UNKNOWNINJURIES_BICYCLIST   331049 non-null  int64              
 9   FATAL_BICYCLIST             331049 non-null  int64              
 10  MAJORINJURIES_DRIVER        331049 non-null  int6

In [7]:
# Coordinates & ward
# Convert LATITUDE and LONGITUDE to numeric, invalid values become NaN
crashes["LATITUDE"] = pd.to_numeric(crashes["LATITUDE"], errors="coerce")
crashes["LONGITUDE"] = pd.to_numeric(crashes["LONGITUDE"], errors="coerce")

# Strip whitespace from WARD (if string)
crashes["WARD"] = crashes["WARD"].astype(str).str.strip()

# Remove 2 rows w/ missing Ward info
crashes = crashes.dropna(subset=["WARD"]).copy()

# --- d. Consolidate injuries/fatalities ---
fatal_cols = ["FATAL_BICYCLIST", "FATAL_DRIVER", "FATAL_PEDESTRIAN", "FATALPASSENGER", "FATALOTHER"]
major_cols = ["MAJORINJURIES_BICYCLIST", "MAJORINJURIES_DRIVER", "MAJORINJURIES_PEDESTRIAN", "MAJORINJURIESPASSENGER", "MAJORINJURIESOTHER"]
minor_cols = ["MINORINJURIES_BICYCLIST", "MINORINJURIES_DRIVER", "MINORINJURIES_PEDESTRIAN", "MINORINJURIESPASSENGER", "MINORINJURIESOTHER"]

# Sum across columns, handling missing columns safely with errors='ignore'
crashes["FATALITIES"] = crashes[fatal_cols].sum(axis=1, skipna=True)
crashes["MAJORINJURIES"] = crashes[major_cols].sum(axis=1, skipna=True)
crashes["MINORINJURIES"] = crashes[minor_cols].sum(axis=1, skipna=True)

# --- e. Human factor flags ---
# Speeding flag
crashes["speeding"] = (crashes.get("SPEEDING_INVOLVED", 0) > 0).astype(int)

# Impaired flag: check columns exist, default to False if missing
impaired_driver = crashes.get("DRIVERSIMPAIRED", 0)
impaired_ped = crashes.get("PEDESTRIANSIMPAIRED", 0)
crashes["impaired"] = ((impaired_driver > 0) | (impaired_ped > 0)).astype(int)

# --- f. Severity category ---
def severity(row):
    if row["FATALITIES"] > 0:
        return "Fatal"
    elif (row["MAJORINJURIES"] + row["MINORINJURIES"]) > 0:
        return "Injury"
    else:
        return "Property Damage Only"

crashes["severity"] = crashes.apply(severity, axis=1)

In [8]:
# Drop the old injury/fatality and human factor columns 
cols_to_drop = [ "REPORTDATE", "FROMDATE",
    "MAJORINJURIES_BICYCLIST", "MINORINJURIES_BICYCLIST", "UNKNOWNINJURIES_BICYCLIST", "FATAL_BICYCLIST",
    "MAJORINJURIES_DRIVER", "MINORINJURIES_DRIVER", "UNKNOWNINJURIES_DRIVER", "FATAL_DRIVER",
    "MAJORINJURIES_PEDESTRIAN", "MINORINJURIES_PEDESTRIAN", "UNKNOWNINJURIES_PEDESTRIAN", "FATAL_PEDESTRIAN",
    "PEDESTRIANSIMPAIRED", "BICYCLISTSIMPAIRED", "DRIVERSIMPAIRED",
    "TOTAL_TAXIS", "TOTAL_GOVERNMENT", "SPEEDING_INVOLVED",
    "FATALPASSENGER", "MAJORINJURIESPASSENGER", "MINORINJURIESPASSENGER", "UNKNOWNINJURIESPASSENGER",
    "MAJORINJURIESOTHER", "MINORINJURIESOTHER", "UNKNOWNINJURIESOTHER", "FATALOTHER"
]

crashes = crashes.drop(columns=cols_to_drop, errors="ignore")

# Rename created columns to lowercase 
crashes = crashes.rename(columns={
    "FATALITIES": "fatalities",
    "MAJORINJURIES": "majorinjuries",
    "MINORINJURIES": "minorinjuries"
})

In [9]:
# Count original rows in 'crashes'
original_count = crashes.shape[0]

# Filter for crashes from 2011 or later
crashes = crashes[crashes["year"] >= 2011].copy()

# Count rows after filtering and print # of rows dropped
new_count = crashes.shape[0]
rows_dropped = original_count - new_count
print(f"Number of rows dropped (pre-2011): {rows_dropped}")
print(f"Remaining rows: {new_count}")

# Create a copy for export 
crashesClean = crashes.copy()

print(crashesClean.info())

print(crashesClean.isna().sum())

Number of rows dropped (pre-2011): 27443
Remaining rows: 303606
<class 'pandas.core.frame.DataFrame'>
Index: 303606 entries, 10 to 338315
Data columns (total 19 columns):
 #   Column                Non-Null Count   Dtype              
---  ------                --------------   -----              
 0   ADDRESS               303606 non-null  object             
 1   LATITUDE              303606 non-null  float64            
 2   LONGITUDE             303606 non-null  float64            
 3   WARD                  303606 non-null  object             
 4   TOTAL_VEHICLES        303606 non-null  int64              
 5   TOTAL_BICYCLES        303606 non-null  int64              
 6   TOTAL_PEDESTRIANS     303606 non-null  int64              
 7   NEARESTINTSTREETNAME  303606 non-null  object             
 8   OFFINTERSECTION       303606 non-null  float64            
 9   crash_datetime        303606 non-null  datetime64[ns, UTC]
 10  year                  303606 non-null  int32            

In [10]:
# Save the cleaned dataframe as CSV
crashesClean.to_csv("crashesClean.csv", index=False)

# Final size of 'crashesClean' --> 303,606 rows x 19 columns
print("crashesClean saved as 'crashesClean.csv', shape:", crashesClean.shape)

crashesClean saved as 'crashesClean.csv', shape: (303606, 19)
